In [54]:
import pandas as pd
import ipywidgets as widgets
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 20)
from ipywidgets import interact, interact_manual

In [55]:
df = pd.read_csv('board.csv', header=None)

In [56]:
output = widgets.Output()
with output:
    display(df.loc[1:])
hbox = widgets.HBox([output])
hbox

def refreshBoard(source=None):
    output.clear_output()
    with output:
        display(board.tail(20))
        
left=widgets.Button(description='LEFT')
left.on_click(refreshBoard)
left

Button(description='LEFT', style=ButtonStyle())

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

In [29]:
@interact
def refresh(dir=(1,3,1)):
    return df

interactive(children=(IntSlider(value=2, description='dir', max=3, min=1), Output()), _dom_classes=('widget-in…

In [88]:
a = input()

1


In [ ]:
from pynput.keyboard import Key, Listener

def on_press(key):
    print('{0} pressed'.format(
        key))

def on_release(key):
    print('{0} release'.format(
        key))
    if key == Key.esc:
        # Stop listener
        return False

# Collect events until released
with Listener(
        on_press=on_press,
        on_release=on_release) as listener:
    listener.join()